<h1 align="center" style="color: #A9A9A9;"> Welcome to the Quantum Chemistry </h1>

# <a id='toc1_'></a>[ ](#toc0_)
---

|$\LARGE \textbf{\textcolor{red}{gdb9: Openbabel xTB-crest and Pyscf Quantum Chemistry Calculations Assessment}}$
|                                                                                                                          
---


 ###  **MVOTO KONGO Patrick Sorrel**, sorrel.mvoto@facsciences-uy1.cm

 
* Department of Physics, Faculty of Science, University of Yaounde I,  Ph.D Candidate
* AtomiC Molecular Physics et Biophysics
  
23 FEVRIER 2025

---

In [1]:
import pandas.util  # Assuming 'util' is an alias for pandas.util
import pandas as pd
# Read the CSV file

# df = pd.read_csv("best_GDB9.csv")
df=pd.read_pickle('filtered_molecules.pkl')
# Filter rows with pce_1 > 10
#df.rename(columns={'Id':'smiles_key','Gap': 'GAP(eV)','HOMO': 'HOMO(eV)','LUMO': 'LUMO(eV)'}, inplace=True)
#df = df.loc[:, ['smiles_key', 'SMILES', 'HOMO(eV)','LUMO(eV)', 'GAP(eV)']]
df

,smiles_key,smiles,HOMO,LUMO,Gap,HOMO(eV),LUMO(eV),Gap(eV),Scharber_Gap(eV)
0,gdb 326,O=CC(=O)C=O,-0.2606,-0.1455,0.1151,-7.091287,-3.959257,3.132031,3.132031
1,gdb 723,C1=NN=CN=N1,-0.2475,-0.1121,0.1355,-6.734818,-3.050396,3.687143,3.687143
2,gdb 1587,CC(=O)C(=O)C=O,-0.2489,-0.1149,0.1340,-6.772914,-3.126588,3.646326,3.646326
3,gdb 1589,NC(=O)C(=O)C=O,-0.2482,-0.1217,0.1265,-6.753866,-3.311626,3.442240,3.442240
4,gdb 2115,O=C1CC=CC1=O,-0.2347,-0.0951,0.1396,-6.386512,-2.587803,3.798709,3.798709
...,...,...,...,...,...,...,...,...,...
519,gdb 133853,OC12C3C1N1CC3N=C21,-0.2087,-0.0889,0.1198,-5.679016,-2.419092,3.259924,3.259924
520,gdb 133854,CC12CC34CC=C(C13)C24,-0.1575,-0.0912,0.0663,-4.285793,-2.481678,1.804115,1.804115
521,gdb 133858,CC1N2C3C4=CCC13C24,-0.1693,-0.1041,0.0652,-4.606888,-2.832705,1.774182,1.774182
522,gdb 133864,C1C2CC3OC4CC1C234,-0.1386,-0.0638,0.0747,-3.771498,-1.736086,2.032691,2.032691


In [2]:
import random

In [3]:
# for reproductible results
random.seed(3)

In [4]:
positions = []
for i in range(6):
    position = random.randrange(len(df))
    positions.append(position)

In [5]:
df = df.loc[df.index.isin(positions)]
df.reset_index(drop=True, inplace=True)
df

,smiles_key,smiles,HOMO,LUMO,Gap,HOMO(eV),LUMO(eV),Gap(eV),Scharber_Gap(eV)
0,gdb 4807,[NH3+]CC1=CC([O-])=CO1,-0.1896,-0.1091,0.0805,-5.159279,-2.968762,2.190517,2.190517
1,gdb 23140,[O-]N=C(CC#C)C1C[NH2+]1,-0.2163,-0.0979,0.1183,-5.885823,-2.663995,3.219107,3.219107
2,gdb 28830,NC1=CC=CC(=O)C=N1,-0.2094,-0.0745,0.1349,-5.698064,-2.027248,3.670816,3.670816
3,gdb 50811,N=C(NC=O)C(=O)C=O,-0.2537,-0.1376,0.1161,-6.903529,-3.744287,3.159242,3.159242
4,gdb 125389,CC1=COC2=C1N=NO2,-0.2170,-0.0804,0.1366,-5.904871,-2.187795,3.717075,3.717075
5,gdb 130686,O=C1NC2=C(ON=N2)O1,-0.2445,-0.1080,0.1365,-6.653184,-2.938830,3.714354,3.714354


In [6]:
!xtb --version

      -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.7.1 (edcfbbe) compiled by 'albert@albert-system' on 2024-07-22

normal termination of xtb


In [7]:
from pyscf.data import nist
import time

#conversion en eV
au2ev = nist.HARTREE2EV

def find_homo_lumo(myhf, au2ev):
    """Function that returns the HOMO and LUMO index and the HOMO energy in eV

    Args:
        mf_pyscf (pyscf object): pyscf meam-field object of the molecule to be evaluated.
    """
    # Index of HOMO and LUMO
    lumo_idx = myhf.mo_occ.tolist().index(0.)
    homo_idx = lumo_idx - 1

    # Calculate the HOMO Homo-LUMO
    E_HOMO = myhf.mo_energy[homo_idx]*au2ev
    E_LUMO = myhf.mo_energy[lumo_idx]*au2ev
    E_g = abs(E_HOMO - E_LUMO)

    return E_HOMO, E_LUMO, E_g

###  <a id='toc1_1_'></a>[Fonction pour le Calcul des Énergies HOMO, LUMO et GAP](#toc0_)


Calcul des énergies `HOMO, LUMO et GAP` avec `PySCF` en utilisant la base `B3LYP/6-31G(2df,p)` via les méthodes `DFT et HF`.


In [8]:

from pyscf import dft,scf,gto
def pyscf_calculation(mol_xtb_xyz, directory):
    
    mol_pyscf = gto.Mole(
        atom=mol_xtb_xyz,
        charge=0,
        spin=0,
        basis="6-31G(2df,p)",
        symmetry=False,
        unit='Angstrom'
    )
    mol_pyscf.build()
    
   # DFT calculation with B3LYP functional
    mdf = dft.RKS(mol_pyscf, xc="B3LYP").run()


    Eks1_homo, Eks1_lumo, Eks1_g = find_homo_lumo(mdf, au2ev)
    
    
    return Eks1_homo, Eks1_lumo, Eks1_g

In [9]:
import pyscf
pyscf.__version__

'2.7.0'

In [10]:

import subprocess,os 
from rdkit import Chem

from rdkit.Chem import AllChem
import rdkit
import subprocess as sp

import re


In [11]:
def log_1error(file_path, process, error_message):
    with open("error_log.txt", "a") as error_file:
        error_file.write(f"Error processing {file_path} with {process}: {error_message}\n")

In [12]:

def log_error(smiles_key, method_gfn, stage, error_message, error_file="error_log.txt"):
    with open(error_file, "a") as f:
        f.write(f"SMILES: {smiles_key}, GFN: {method_gfn}, Stage: {stage}, Error: {error_message}\n")

###  <a id='toc1_1_'></a>[Fonction pour le Calcul XTB : Optimisation et Pré-optimisation](#toc0_)

Calcul d'optimisation et pré-optimisation avec `XTB` en utilisant les modèles `GFN2, GFN1, et GFN0`.

In [13]:
def clean_xtb_files1():
    #---------------------------------------------------------------------
        # Clean up output files from CREST processes
        #FIXME To call after each xtb function
        #---------------------------------------------------------------------
        sp.run(['rm', 'bondlengths', 'charges', 'coord', 'coord.original', 'cregen_0.tmp',
                'cregen_1.tmp', 'cre_members', 'crest_best.xyz', 'crest_conformers.xyz',
                'crest.energies', 'crest_rotamers.xyz', 'gfnff_charges', 'gfnff_topo',
                '.history.xyz', 'wbo', 'crest_property.xyz', 'gfnff_adjacency', '.UHF',
                'ensemble_energies.log', 'charges3', 'charges', 'molden.input', 'crest_0.mdrestart',
                'crest_dynamics.trj', 'crestopt.log', 'crest.restart','crest.engrad','crestopt.xyz', 'crest_input_copy.xyz','g98.out'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
        # For folder
        sp.run(['rm', '-r', 'calculation.level.1'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)


In [14]:
def clean_xtb_files():
    #---------------------------------------------------------------------
    # Clean up output files from xtb
    #FIXME To call after running crest function and leave crest_bestxyz file
    #---------------------------------------------------------------------
    sp.run(['rm', 'bondlengths', 'charges', 'coord', 'coord.original' , 'vibspectrum', 'hessian', 'gfnff_charges', 'gfnff_topo', 'wfn.xtb', 'xtbhess.xyz',
            '.history.xyz', 'struc.xyz', 'wbo', 'xtbopt.xyz', 'xtbopt.log', '.xtboptok','g98.out',
            'xtbrestart', 'xtbtopo.mol', 'xtblast.xyz', 'gfnff_adjacency', '.UHF',
            'ensemble_energies.log', 'charges3', 'charges', 'molden.input','test.smi','pat.xyz','sample.sdf'],
           stdout=sp.DEVNULL, stderr=sp.DEVNULL)
    # For folder
    sp.run(['rm', '-r', 'calculation.level.1', 'PROP'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)

In [15]:
def generate_3d_conformation(smiles_key, smiles, working_dir,path_xyz):
    """
    Generate a 3D conformation using Open Babel from SMILES and store the coordinates in a file.
    """
    # Define path for the output XYZ file
    path_xyz = working_dir / f'{smiles_key}.xyz'

    # Check if the file already exists
    if not path_xyz.exists():
        try:
            # Execute the Open Babel command
                # optimisation avec openbabel

            with open('test.smi', 'w') as f:
                f.writelines([smiles])

            OB_process= sp.run('obabel  test.smi -O  pat.xyz --gen3d -h --best --errorlevel 2 --minimize --ff MMFF94s  --steps 15000 --sd --crit 1e-9 --log', shell=True, stdout=sp.PIPE, stderr=sp.PIPE, universal_newlines=True, text=True)
            sp.run(['cp', 'pat.xyz', str(path_xyz)], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
            print(OB_process)
            return path_xyz
        except sp.CalledProcessError as e:
            print(f"An error occurred while processing molecule {smiles_key}: {e}")
        return None

In [16]:
def get_xtb_energy_1(path_xtb1_opt_log, smiles_key, working_dir):
        path_xtb1_opt_log = working_dir / f'{smiles_key}_xtb1_opt.log'

        if path_xtb1_opt_log.exists():
            with open(path_xtb1_opt_log, 'r') as f:
                text_content = f.readlines()

            # Read the output (implementation details omitted)

            output_index = [i for i in range(len(text_content)) if 'Property Printout' in text_content[i]]
            text_content = text_content[output_index[0]:]
            homo_data = [x for x in text_content if '(HOMO)' in x]
            lumo_data = [x for x in text_content if '(LUMO)' in x]
            homo_lumo_gap = [x for x in text_content if 'HOMO-LUMO GAP' in x]
            total = [x for x in text_content if 'TOTAL ENERGY' in x]
            total0= float(total[0].split(' ')[-5])
            total1=total0*au2ev
            total2=total0*627.509474
            lumo_val = float(lumo_data[0].split(' ')[-2])
            homo_val = float(homo_data[0].split(' ')[-2])
            homo_lumo_val = float(homo_lumo_gap[0].split(' ')[-5])

             # Write the properties to a single file (modify as needed)
            with open(os.path.join(working_dir, f'{smiles_key}_properties.txt'), 'a') as f:
                 f.write(f'LUMO: {lumo_val}\n')
                 f.write(f'HOMO: {homo_val}\n')
                 f.write(f'HOMO-LUMO GAP: {homo_lumo_val}\n')

        return  homo_lumo_val, homo_val, lumo_val

In [17]:
import pandas as pd
import subprocess as sp
import time
from pathlib import Path

import os
import subprocess as sp
import time
from pathlib import Path
import pandas as pd

def run_xtb_process(path_xyz, output_xyz, log_path, path_output_sdf, smiles_key, method_gfn):
    try:
        start_xtb = time.time()
        xtb_process = sp.run(
            ["xtb", str(path_xyz) ,"--opt", "vtight", "--gfn",str(method_gfn), "--alpb", "toluene", "--chrg", "0"],
            stdout=sp.PIPE, stderr=sp.PIPE, text=True, check=True
        )
        xtb_time = time.time() - start_xtb
        
        if os.path.exists("xtbopt.xyz"):
            sp.run(['cp', 'xtbopt.xyz', str(output_xyz)], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
        if os.path.exists("xtbopt.xyz"):
            sp.run(['obabel', 'xtbopt.xyz', '-O', str(path_output_sdf)], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
        
        with open(log_path, "w") as fl:
            fl.write(xtb_process.stdout)
        return xtb_time, output_xyz, log_path
    except sp.CalledProcessError as e:
        log_error(smiles_key, method_gfn, "xtb", str(e))
        return None, None, None

def run_crest_process(path_xyz, output_xyz, log_path, smiles_key, method_gfn):
    try:
        start_crest = time.time()
        crest_process = sp.run(
            ["crest", str(path_xyz), "--gfn", str(method_gfn), "--mquick", "--noreftopo", "--opt", "vtight", "--alpb", "toluene"],
            stdout=sp.PIPE, stderr=sp.PIPE, text=True, check=True
        )
        crest_time = time.time() - start_crest
        
        if os.path.exists("crestopt.xyz"):
            sp.run(['cp', 'crestopt.xyz', str(output_xyz)], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
        
        with open(log_path, "w") as fl1:
            fl1.write(crest_process.stdout)
        return crest_time, output_xyz
    except sp.CalledProcessError as e:
        log_error(smiles_key, method_gfn, "crest", str(e))
        return None, None

def calculate_properties_xtb_crest(df, working_dir):
    """
    Évalue l'énergie avec xTB sur les coordonnées xyz générées pour chaque entrée du DataFrame.
    
    Paramètres:
    df : Pandas DataFrame contenant les colonnes 'SMILES' et 'smiles_key'.
    working_dir : Répertoire où tous les fichiers seront sauvegardés et traités.
    
    Retourne:
    df_ENERGY : DataFrame contenant les propriétés calculées.
    """
    columns = ["smiles_key", "SMILES", "HOMO(eV)", "HOMO_xtb(eV)", "HOMO_DFT(eV)", "LUMO(eV)", 
               "LUMO_xtb(eV)", "LUMO_DFT(eV)", "GAP(eV)", "GAP_xtb(eV)", "GAP_DFT(eV)"]
    dtb = []
    results = []
    df = df.copy()
    
    working_dir = Path(working_dir) if isinstance(working_dir, str) else working_dir
    working_dir.mkdir(parents=True, exist_ok=True)
    
    for _, row in df.iterrows():
        smiles_key = row["smiles_key"]
        smiles = row["smiles"]
        HOMO, LUMO, GAP = row["HOMO(eV)"], row["LUMO(eV)"], row["Gap(eV)"]
        
        paths = {key: working_dir / f"{smiles_key}_{key}.xyz" for key in ["xyz", "xtb1_pre_opt", "xtb1_opt", "crest1_opt"]}
        logs = {key: working_dir / f"{smiles_key}_{key}.log" for key in ["xtb1_pre_opt", "crest1", "xtb1_opt"]}
        sdf_paths = {key: working_dir / f"{smiles_key}_{key}.sdf" for key in ["xtb_pre_opt", "xtb_opt"]}
        
        # Génération de la structure 3D
        result = generate_3d_conformation(smiles_key, smiles, working_dir, paths["xyz"])
        if result is None:
            log_error(smiles_key, "N/A", "3D generation", "Failed to generate 3D structure")
            continue
        
        method_gfn = '2'
        retry_methods = ['2', '1', '0']
        
        for method in retry_methods:
            xtb_result = run_xtb_process(result, paths["xtb1_pre_opt"], logs["xtb1_pre_opt"], sdf_paths["xtb_pre_opt"], smiles_key, method)
            print(xtb_result)
            clean_xtb_files()
            if xtb_result[0] is not None:
                crest_result = run_crest_process(paths["xtb1_pre_opt"], paths["crest1_opt"], logs["crest1"], smiles_key, method)
                clean_xtb_files1()
                if crest_result[0] is not None:
                    break
                log_error(smiles_key, method, "conformer search", "Topological changes detected or CREST failed")
            else:
                log_error(smiles_key, method, "preoptimization", "Topological changes detected or xTB failed")
        else:
             continue
        # final_xtb_result = run_xtb_process(paths["crest1_opt"],paths["xtb1_opt"], paths["xtb1_opt"], logs["xtb1_opt"], sdf_paths["xtb_opt"], smiles_key, method)
        # if final_xtb_result[0] is None or not os.path.exists(paths["xtb1_opt"]):
        #     log_error(smiles_key, method_gfn, "optimization", "Topological changes detected or xTB failed")
        #     continue
        # clean_xtb_files()
        final_xtb_result = run_xtb_process(paths["crest1_opt"], paths["xtb1_opt"], logs["xtb1_opt"], sdf_paths["xtb_opt"], smiles_key, method)
        if final_xtb_result[0] is None or not os.path.exists(paths["xtb1_opt"]):
            log_error(smiles_key, method_gfn, "optimization", "Topological changes detected or xTB failed")
            continue
        clean_xtb_files()
        
        with open(paths["xtb1_opt"], 'r') as f:
            lines = f.readlines()
        
        if len(lines) < 3:
            log_error(smiles_key, method_gfn, "file_read", "xtb1_opt.xyz file is empty or corrupted")
            continue
        
        mol_xtb1_xyz = '\n'.join(lines[2:])
        
        Eks1_homo, Eks1_lumo, Eks1_g = pyscf_calculation(mol_xtb1_xyz, working_dir)
        homo_lumo_val, homo_val, lumo_val = get_xtb_energy_1(logs["xtb1_opt"], smiles_key, working_dir)

        results.append([
            smiles_key, smiles, HOMO, homo_val, Eks1_homo, LUMO, lumo_val, Eks1_lumo, GAP, homo_lumo_val, Eks1_g
        ])
    
    return pd.DataFrame(results, columns=columns)

MY_crest1_GDB9 = Path(os.getcwd()) / 'MY_crest1_GDB9'
MY_crest1_GDB9.mkdir(exist_ok=True)
df_ENERGY1 = calculate_properties_xtb_crest(df, MY_crest1_GDB9)
df_ENERGY1.to_pickle('dataset_for_ml.pkl')



MY_crest1_GDB9 = Path(os.getcwd()) / 'MY_crest1_GDB9'
MY_crest1_GDB9.mkdir(exist_ok=True)
df_ENERGY1 = calculate_properties_xtb_crest(df, MY_crest1_GDB9)
df_ENERGY1.to_pickle('dataset_for_ml.pkl')


df_ENERGY1

In [ ]:


import pandas.util  # Assuming 'util' is an alias for pandas.util
import pandas as pd

df=pd.read_pickle('filtered_molecules.pkl')



from pyscf.data import nist
import time

#conversion en eV
au2ev = nist.HARTREE2EV

def find_homo_lumo(myhf, au2ev):
    """Function that returns the HOMO and LUMO index and the HOMO energy in eV

    Args:
        mf_pyscf (pyscf object): pyscf meam-field object of the molecule to be evaluated.
    """
    # Index of HOMO and LUMO
    lumo_idx = myhf.mo_occ.tolist().index(0.)
    homo_idx = lumo_idx - 1

    # Calculate the HOMO Homo-LUMO
    E_HOMO = myhf.mo_energy[homo_idx]*au2ev
    E_LUMO = myhf.mo_energy[lumo_idx]*au2ev
    E_g = abs(E_HOMO - E_LUMO)

    return E_HOMO, E_LUMO, E_g


from pyscf import dft,scf,gto
def pyscf_calculation(mol_xtb_xyz, directory):

    mol_pyscf = gto.Mole(
        atom=mol_xtb_xyz,
        charge=0,
        spin=0,
        basis="6-31G(2df,p)",
        symmetry=True,
        unit='Angstrom'
    )
    mol_pyscf.build()

   # DFT calculation with B3LYP functional
    mdf = dft.RKS(mol_pyscf, xc="B3LYP").run()


    Eks1_homo, Eks1_lumo, Eks1_g = find_homo_lumo(mdf, au2ev)


    return Eks1_homo, Eks1_lumo, Eks1_g


import os
from rdkit import Chem

from rdkit.Chem import AllChem
import rdkit
import subprocess as sp

import re

def log_1error(file_path, process, error_message):
    with open("error_log.txt", "a") as error_file:
        error_file.write(f"Error processing {file_path} with {process}: {error_message}\n")

def log_error(smiles_key, method_gfn, stage, error_message, error_file="error_log.txt"):
    with open(error_file, "a") as f:
        f.write(f"SMILES: {smiles_key}, GFN: {method_gfn}, Stage: {stage}, Error: {error_message}\n")

"""###  <a id='toc1_1_'></a>[Fonction pour le Calcul XTB : Optimisation et Pré-optimisation](#toc0_)

Calcul d'optimisation et pré-optimisation avec `XTB` en utilisant les modèles `GFN2, GFN1, et GFN0`.
"""

def clean_xtb_files1():
    #---------------------------------------------------------------------
        # Clean up output files from CREST processes
        #FIXME To call after each xtb function
        #---------------------------------------------------------------------
        sp.run(['rm', 'bondlengths', 'charges', 'coord', 'coord.original', 'cregen_0.tmp',
                'cregen_1.tmp', 'cre_members', 'crest_best.xyz', 'crest_conformers.xyz',
                'crest.energies', 'crest_rotamers.xyz', 'gfnff_charges', 'gfnff_topo',
                '.history.xyz', 'wbo', 'crest_property.xyz', 'gfnff_adjacency', '.UHF',
                'ensemble_energies.log', 'charges3', 'charges', 'molden.input', 'crest_0.mdrestart',
                'crest_dynamics.trj', 'crestopt.log', 'crest.restart','crest.engrad','crestopt.xyz', 'crest_input_copy.xyz','g98.out'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
        # For folder
        sp.run(['rm', '-r', 'calculation.level.1'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)

def clean_xtb_files():
    #---------------------------------------------------------------------
    # Clean up output files from xtb
    #FIXME To call after running crest function and leave crest_bestxyz file
    #---------------------------------------------------------------------
    sp.run(['rm', 'bondlengths', 'charges', 'coord', 'coord.original' , 'vibspectrum', 'hessian', 'gfnff_charges', 'gfnff_topo', 'wfn.xtb', 'xtbhess.xyz',
            '.history.xyz', 'struc.xyz', 'wbo', 'xtbopt.xyz', 'xtbopt.log', '.xtboptok','g98.out',
            'xtbrestart', 'xtbtopo.mol', 'xtblast.xyz', 'gfnff_adjacency', '.UHF',
            'ensemble_energies.log', 'charges3', 'charges', 'molden.input','test.smi','pat.xyz','sample.sdf'],
           stdout=sp.DEVNULL, stderr=sp.DEVNULL)
    # For folder
    sp.run(['rm', '-r', 'calculation.level.1', 'PROP'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)

def generate_3d_conformation(smiles_key, smiles, working_dir,path_xyz):
    """
    Generate a 3D conformation using Open Babel from SMILES and store the coordinates in a file.
    """
    # Define path for the output XYZ file
    path_xyz = working_dir / f'{smiles_key}.xyz'

    # Check if the file already exists
    if not path_xyz.exists():
        try:
            # Execute the Open Babel command
                # optimisation avec openbabel

            with open('test.smi', 'w') as f:
                f.writelines([smiles])

            OB_process= sp.run('obabel  test.smi -O  pat.xyz --gen3d -h --best --errorlevel 2 --minimize --ff MMFF94s  --steps 15000 --sd --crit 1e-9 --log', shell=True, stdout=sp.PIPE, stderr=sp.PIPE, universal_newlines=True, text=True)
            sp.run(['cp', 'pat.xyz', str(path_xyz)], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
            return path_xyz
        except sp.CalledProcessError as e:
            print(f"An error occurred while processing molecule {smiles_key}: {e}")
        return None

def get_xtb_energy_1(path_xtb1_opt_log, smiles_key, working_dir):
        path_xtb1_opt_log = working_dir / f'{smiles_key}_xtb1_opt.log'

        if path_xtb1_opt_log.exists():
            with open(path_xtb1_opt_log, 'r') as f:
                text_content = f.readlines()

            # Read the output (implementation details omitted)

            output_index = [i for i in range(len(text_content)) if 'Property Printout' in text_content[i]]
            text_content = text_content[output_index[0]:]
            homo_data = [x for x in text_content if '(HOMO)' in x]
            lumo_data = [x for x in text_content if '(LUMO)' in x]
            homo_lumo_gap = [x for x in text_content if 'HOMO-LUMO GAP' in x]
            total = [x for x in text_content if 'TOTAL ENERGY' in x]
            total0= float(total[0].split(' ')[-5])
            total1=total0*au2ev
            total2=total0*627.509474
            lumo_val = float(lumo_data[0].split(' ')[-2])
            homo_val = float(homo_data[0].split(' ')[-2])
            homo_lumo_val = float(homo_lumo_gap[0].split(' ')[-5])

             # Write the properties to a single file (modify as needed)
            with open(os.path.join(working_dir, f'{smiles_key}_properties.txt'), 'a') as f:
                 f.write(f'LUMO: {lumo_val}\n')
                 f.write(f'HOMO: {homo_val}\n')
                 f.write(f'HOMO-LUMO GAP: {homo_lumo_val}\n')

        return  homo_lumo_val, homo_val, lumo_val


import time
from pathlib import Path


def run_xtb_process(path_xyz, output_xyz, log_path, path_output_sdf, smiles_key, method_gfn,charge):
    try:
        start_xtb = time.time()
        xtb_process = sp.run(
            ["xtb", str(path_xyz) ,"--opt", "vtight", "--gfn",str(method_gfn), "--chrg",str(charge),"--parallel", "4", "--alpb", "toluene"],
            stdout=sp.PIPE, stderr=sp.PIPE, text=True, check=True
        )
        xtb_time = time.time() - start_xtb

        if os.path.exists("xtbopt.xyz"):
            sp.run(['cp', 'xtbopt.xyz', str(output_xyz)], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
        if os.path.exists("xtbopt.xyz"):
            sp.run(['obabel', 'xtbopt.xyz', '-O', str(path_output_sdf)], stdout=sp.DEVNULL, stderr=sp.DEVNULL)

        with open(log_path, "w") as fl:
            fl.write(xtb_process.stdout)
        return xtb_time, output_xyz, log_path
    except sp.CalledProcessError as e:
        log_error(smiles_key, method_gfn, "xtb", str(e))
        return None, None, None

def run_crest_process(path_xyz, output_xyz, log_path, smiles_key, method_gfn,charge):
    try:
        start_crest = time.time()
        crest_process = sp.run(
            ["crest", str(path_xyz), "--gfn", str(method_gfn), "--mquick", "--noreftopo", "--opt", "vtight","--chrg",str(charge),"--parallel", "4", "--alpb", "toluene"],
            stdout=sp.PIPE, stderr=sp.PIPE, text=True, check=True
        )
        crest_time = time.time() - start_crest

        if os.path.exists("crestopt.xyz"):
            sp.run(['cp', 'crestopt.xyz', str(output_xyz)], stdout=sp.DEVNULL, stderr=sp.DEVNULL)

        with open(log_path, "w") as fl1:
            fl1.write(crest_process.stdout)
        return crest_time, output_xyz
    except sp.CalledProcessError as e:
        log_error(smiles_key, method_gfn, "crest", str(e))
        return None, None

# def calculate_properties_xtb_crest(df, working_dir):
#     """
#     Évalue l'énergie avec xTB sur les coordonnées xyz générées pour chaque entrée du DataFrame.

#     Paramètres:
#     df : Pandas DataFrame contenant les colonnes 'SMILES' et 'smiles_key'.
#     working_dir : Répertoire où tous les fichiers seront sauvegardés et traités.

#     Retourne:
#     df_ENERGY : DataFrame contenant les propriétés calculées.
#     """
#     columns = ["smiles_key", "SMILES", "HOMO(eV)", "HOMO_xtb(eV)", "HOMO_DFT(eV)", "LUMO(eV)",
#                "LUMO_xtb(eV)", "LUMO_DFT(eV)", "GAP(eV)", "GAP_xtb(eV)", "GAP_DFT(eV)"]
#     dtb = []
#     results = []
#     df = df.copy()

#     working_dir = Path(working_dir) if isinstance(working_dir, str) else working_dir
#     working_dir.mkdir(parents=True, exist_ok=True)

#     for _, row in df.iterrows():
#         smiles_key = row["smiles_key"]
#         smiles = row["smiles"]
#         HOMO, LUMO, GAP = row["HOMO(eV)"], row["LUMO(eV)"], row["Gap(eV)"]

#         paths = {key: working_dir / f"{smiles_key}_{key}.xyz" for key in ["xyz", "xtb1_pre_opt", "xtb1_opt", "crest1_opt"]}
#         logs = {key: working_dir / f"{smiles_key}_{key}.log" for key in ["xtb1_pre_opt", "crest1", "xtb1_opt"]}
#         sdf_paths = {key: working_dir / f"{smiles_key}_{key}.sdf" for key in ["xtb_pre_opt", "xtb_opt"]}

#         # Génération de la structure 3D
#         result = generate_3d_conformation(smiles_key, smiles, working_dir, paths["xyz"])
#         if result is None:
#             log_error(smiles_key, "N/A", "3D generation", "Failed to generate 3D structure")
#             continue

#         method_gfn = '2'
#         retry_methods = ['2', '1', '0']

#         for method in retry_methods:
#             xtb_result = run_xtb_process(result, paths["xtb1_pre_opt"], logs["xtb1_pre_opt"], sdf_paths["xtb_pre_opt"], smiles_key, method)

#             clean_xtb_files()
#             if xtb_result[0] is not None:
#                 crest_result = run_crest_process(paths["xtb1_pre_opt"], paths["crest1_opt"], logs["crest1"], smiles_key, method)
#                 clean_xtb_files1()
#                 if crest_result[0] is not None:
#                     break
#                 log_error(smiles_key, method, "conformer search", "Topological changes detected or CREST failed")
#             else:
#                 log_error(smiles_key, method, "preoptimization", "Topological changes detected or xTB failed")
#         else:
#              continue
#         # final_xtb_result = run_xtb_process(paths["crest1_opt"],paths["xtb1_opt"], paths["xtb1_opt"], logs["xtb1_opt"], sdf_paths["xtb_opt"], smiles_key, method)
#         # if final_xtb_result[0] is None or not os.path.exists(paths["xtb1_opt"]):
#         #     log_error(smiles_key, method_gfn, "optimization", "Topological changes detected or xTB failed")
#         #     continue
#         # clean_xtb_files()
#         final_xtb_result = run_xtb_process(paths["crest1_opt"], paths["xtb1_opt"], logs["xtb1_opt"], sdf_paths["xtb_opt"], smiles_key, method)
#         if final_xtb_result[0] is None or not os.path.exists(paths["xtb1_opt"]):
#             log_error(smiles_key, method_gfn, "optimization", "Topological changes detected or xTB failed")
#             continue
#         clean_xtb_files()

#         with open(paths["xtb1_opt"], 'r') as f:
#             lines = f.readlines()

#         if len(lines) < 3:
#             log_error(smiles_key, method_gfn, "file_read", "xtb1_opt.xyz file is empty or corrupted")
#             continue

#         mol_xtb1_xyz = '\n'.join(lines[2:])

#         Eks1_homo, Eks1_lumo, Eks1_g = pyscf_calculation(mol_xtb1_xyz, working_dir)
#         homo_lumo_val, homo_val, lumo_val = get_xtb_energy_1(logs["xtb1_opt"], smiles_key, working_dir)

#         results.append([
#             smiles_key, smiles, HOMO, homo_val, Eks1_homo, LUMO, lumo_val, Eks1_lumo, GAP, homo_lumo_val, Eks1_g
#         ])

#     return pd.DataFrame(results, columns=columns)
import pandas as pd
from pathlib import Path
import gc
import os

def calculate_properties_xtb_crest(df, working_dir):
    """
    Évalue l'énergie avec xTB sur les coordonnées xyz générées pour chaque entrée du DataFrame,
    en traitant les données par lots de 100 molécules pour éviter les problèmes de mémoire.

    Paramètres:
    df : Pandas DataFrame contenant les colonnes 'SMILES' et 'smiles_key'.
    working_dir : Répertoire où tous les fichiers seront sauvegardés et traités.

    Retourne:
    df_ENERGY : DataFrame contenant les propriétés calculées.
    """
    # Définition des colonnes pour le résultat final
    columns = ["smiles_key", "SMILES", "HOMO(eV)", "HOMO_xtb(eV)", "HOMO_DFT(eV)", 
               "LUMO(eV)", "LUMO_xtb(eV)", "LUMO_DFT(eV)", "GAP(eV)", "GAP_xtb(eV)", "GAP_DFT(eV)"]
    results = []
    df = df.copy()

    # Créer le répertoire de travail s'il n'existe pas
    working_dir = Path(working_dir) if isinstance(working_dir, str) else working_dir
    working_dir.mkdir(parents=True, exist_ok=True)

    # Traitement par lots (100 molécules par lot)
    batch_size = 100
    num_batches = (len(df) + batch_size - 1) // batch_size  # Calcul du nombre total de lots

    for batch_index in range(num_batches):
        # Sélection du lot courant
        batch_df = df.iloc[batch_index * batch_size:(batch_index + 1) * batch_size]

        for _, row in batch_df.iterrows():
            smiles_key = row["smiles_key"]
            smiles = row["smiles"]
            HOMO, LUMO, GAP = row["HOMO(eV)"], row["LUMO(eV)"], row["Gap(eV)"]
            mol = Chem.MolFromSmiles(smiles)
            mol = Chem.AddHs(mol)
            if mol == None: 
                print("INVALID")
        
            charge = Chem.rdmolops.GetFormalCharge(mol)
            atom_number = mol.GetNumAtoms()
            # Définition des chemins pour les fichiers
            paths = {key: working_dir / f"{smiles_key}_{key}.xyz" for key in ["xyz", "xtb1_pre_opt", "xtb1_opt", "crest1_opt"]}
            logs = {key: working_dir / f"{smiles_key}_{key}.log" for key in ["xtb1_pre_opt", "crest1", "xtb1_opt"]}
            sdf_paths = {key: working_dir / f"{smiles_key}_{key}.sdf" for key in ["xtb_pre_opt", "xtb_opt"]}

            # Génération de la structure 3D
            result = generate_3d_conformation(smiles_key, smiles, working_dir, paths["xyz"])
            if result is None:
                log_error(smiles_key, "N/A", "3D generation", "Failed to generate 3D structure")
                continue

            # Essayer plusieurs méthodes pour xTB et CREST
            retry_methods = ['2', '1', '0']
            for method in retry_methods:
                xtb_result = run_xtb_process(result, paths["xtb1_pre_opt"], logs["xtb1_pre_opt"], sdf_paths["xtb_pre_opt"], smiles_key, method,charge)
                clean_xtb_files()

                if xtb_result[0] is not None:
                    crest_result = run_crest_process(paths["xtb1_pre_opt"], paths["crest1_opt"], logs["crest1"], smiles_key, method,charge)
                    clean_xtb_files1()
                    if crest_result[0] is not None:
                        break
                    log_error(smiles_key, method, "conformer search", "Topological changes detected or CREST failed")
                else:
                    log_error(smiles_key, method, "preoptimization", "Topological changes detected or xTB failed")
            else:
                continue

            # Optimisation finale avec xTB
            final_xtb_result = run_xtb_process(paths["crest1_opt"], paths["xtb1_opt"], logs["xtb1_opt"], sdf_paths["xtb_opt"], smiles_key, method,charge)
            if final_xtb_result[0] is None or not os.path.exists(paths["xtb1_opt"]):
                log_error(smiles_key, method, "optimization", "Topological changes detected or xTB failed")
                continue
            clean_xtb_files()

            # Lire les résultats optimisés
            with open(paths["xtb1_opt"], 'r') as f:
                lines = f.readlines()

            if len(lines) < 3:
                log_error(smiles_key, method, "file_read", "xtb1_opt.xyz file is empty or corrupted")
                continue

            mol_xtb1_xyz = '\n'.join(lines[2:])
            Eks1_homo, Eks1_lumo, Eks1_g = pyscf_calculation(mol_xtb1_xyz, working_dir)
            homo_lumo_val, homo_val, lumo_val = get_xtb_energy_1(logs["xtb1_opt"], smiles_key, working_dir)

            # Ajouter les résultats au tableau
            results.append([
                smiles_key, smiles, HOMO, homo_val, Eks1_homo, LUMO, lumo_val, Eks1_lumo, GAP, homo_lumo_val, Eks1_g
            ])

        # Forcer la libération de la mémoire après chaque lot
        gc.collect()

    # Retourner le DataFrame final contenant les résultats
    return pd.DataFrame(results, columns=columns)

MY_crest1_GDB9 = Path(os.getcwd()) / 'MY_crest1_GDB9'
MY_crest1_GDB9.mkdir(exist_ok=True)
df_ENERGY1 = calculate_properties_xtb_crest(df, MY_crest1_GDB9)
df_ENERGY1.to_pickle('dataset_for_ml.pkl')





/home/mvotokps/anaconda3/envs/Patrick/lib/python3.10/site-packages/pyscf/dft/libxc.py:507: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


converged SCF energy = -380.46298093749
converged SCF energy = -396.534314054169
converged SCF energy = -343.36058556861
converged SCF energy = -360.605799296291
converged SCF energy = -380.467278573032
converged SCF energy = -431.982455605873
converged SCF energy = -394.922957917466
converged SCF energy = -394.882024715472
converged SCF energy = -414.7691558496
converged SCF energy = -394.888982801791
converged SCF energy = -414.741995660749
converged SCF energy = -399.825845290304
converged SCF energy = -398.715193366419
converged SCF energy = -414.698328785808
converged SCF energy = -401.122674390477
converged SCF energy = -417.205005058561
converged SCF energy = -399.919486664458
converged SCF energy = -417.276242056431
converged SCF energy = -454.438021279145
converged SCF energy = -415.961906602914
converged SCF energy = -394.883820782433
converged SCF energy = -419.780525234061
converged SCF energy = -415.961538841592
converged SCF energy = -435.847734293562
converged SCF energy